In [1]:
%matplotlib notebook
%pylab

Using matplotlib backend: nbAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
%reload_ext autoreload
%autoreload 2

# The acoustics equation (or two-way wave equation)

<hr style="border-width:4px; border-color:coral"></hr>

The acoustics equations is one of the simplest examples of a hyperbolic system of PDEs.  In general, a hyperbolic system can be written as a matrix system, where  the eigenvalues of the $n \times n$ matrix $A$ are all real, and that the eigenvectors span $\mathcal R^n$.  These $n$ eigenvectors $\mathbf r^p$ can be interpreted as "waves", which move with speeds given by their associated eigenvalues $\lambda^p$.  

For the wave propagation algorithm, we need to define a Riemann solver that can solve for "wave", "speeds" and fluctuations.  Before we can do this, however, you have to have a basic understanding of the structure of the wave equation. The problem below should give you the understanding needed to write the Riemann solver for the WPA. 


## The two way wave equation
<hr style="border-width:4px; border-color:coral"></hr>


The two-way wave equation can be used to model acoustic waves (e.g. sound waves or pressure waves) in air.  In this case, the wave equation is written in terms of pressure $p(x,t)$ ($N/m^2$) and is given by 

\begin{equation}
p_{tt} = c^2 p_{xx} = 0, \qquad -\infty < x < \infty
\end{equation}

where $c$ ($m/s$) is the speed of sound in the air.  A second quantity $u(x,t)$ ($m/s$) is the velocity of a parcel of air as it is perturbed by the pressure wave. 

A model for both the pressure and velocity can be written as a system of first order equations (e.g. containing first derivatives only), given by

\begin{eqnarray}
p_t + \rho c^2 u_x & = & 0 \\
\rho u_t + p_x & = & 0
\end{eqnarray}

where $\rho$ ($kg/m^3$) is the constant density of air.  This can be written as a *quasi-linear* system

\begin{equation}
\begin{pmatrix} p \\ u \end{pmatrix}_t + 
\begin{pmatrix} 0 &  \rho c^2 \\ \frac{1}{\rho} & 0 \end{pmatrix} 
\begin{pmatrix} p \\ u \end{pmatrix}_x =  
\begin{pmatrix} 0 \\ 0 \end{pmatrix}
\end{equation}

or more generally as
\begin{equation}
\mathbf q_t + A \mathbf q_x = 0
\end{equation}

where $\mathbf q(x,t) = (p(x,t), u(x,t))$ and $A$ is the $2 \times 2$ matrix

\begin{equation}
A = \begin{pmatrix} 0 &  \rho c^2 \\ \frac{1}{\rho} & 0 \end{pmatrix}.
\end{equation}


Analytically, we can solve this system in the following steps. 

## Problem : D'Alembert solution to the wave equation
<hr style="border-width:4px; border-color:coral"></hr>

Carry out the steps below to get the classic "D'Alembert solution" to the wave equation.

### Step 1

Find the eigenvalues $\lambda^1$ and $\lambda^2$ and eigenvectors $\mathbf r^1$ and $\mathbf r^2$ of the matrix $A$.  Order your eigenvalues so that
$\lambda^1 < \lambda^2$ and normalize your eigenvectors so that the first component of 
each eigenvector is 1. 

### Step 2
Using the eigen-decomposition of $A$ you found above,  write the vector equation for $\mathbf q$ as a de-coupled system of two "1-way" wave equations in terms of new 
variables $w^1(x,t)$ and $w^2(x,t)$, given by

<br/>

\begin{equation}
\begin{aligned}
w^1_t + \lambda^1 w^1_x & = 0 \\
w^2_t + \lambda^2 w^2_x & = 0
\end{aligned}
\label{eqn:oneway}
\end{equation}

The variables  $w^1(x,t)$ and $w^2(x,t)$ are known as *characteristic variables*. 

**Hint:** Write $A = R\Lambda R^{-1}$, where $R$ is a $2 \times 2$ matrix 
$R = [\mathbf r^1, \;  \mathbf r^2]$ and $\Lambda$ is the $2 \times 2$ diagonal matrix 
$\mbox{diag}(\lambda^1, \lambda^2)$.  Set $\mathbf w = R^{-1} \mathbf q$ and obtain a vector equation for $\mathbf w = (w^1(x,t), w^2(x,t))$. 

### Step 3

Show that the solutions to the 1-way wave equations in Step 2 are given by

<br/>

\begin{equation*}
\begin{aligned}
w^1(x,t) & = w^1(x-\lambda^1 t,0) \\
w^2(x,t) & = w^2(x-\lambda^2 t,0) \\
\end{aligned}
\label{eqn:charvar}
\end{equation*}

### Step 4

Show that the solution $\mathbf q$ is given by

<br/>

\begin{equation}
\begin{pmatrix} p(x,t) \\ u(x,t) \end{pmatrix} = 
w^1(x-\lambda^1 t,0) \mathbf r^1 + w^2(x-\lambda^2 t,0) \mathbf r^2
\label{eqn:solution}
\end{equation}

### Step 5
Show that the solution in Step 4 for $p(x,t)$ is exactly the D'Alembert solution

<br/>

\begin{equation}
p(x,t) = \frac{1}{2}\left(f(x+ct) + f(x-ct)\right)+ 
\frac{1}{2c}\int_{x-ct}^{x+ct} g(s) \; ds
\end{equation}

where $f(x) = p(x,0)$ is the initial pressure field and $g(x) = p_t(x,0)$ is initial rate of change of pressure.  

**Hint:**  Use the definition $\mathbf w = R^{-1} \mathbf q$ in 
Step 4.  Then use the first equation in  to show that the solution you get can be interpreted as a d'Alembert solution with non-zero initial conditions $g(x)$. 

### Step 6

Use the solution in Step 4 to show that $u(x,t)$ is also a d'Alembert solution. 

## Problem : Propagation of an acoustic disturbance
<hr style="border-width:4px; border-color:coral"></hr>


When we hear a sound from a distant source, the "sound" we hear is a perturbation to the background pressure field.  This sound is initiated by the air parcels being instantaneously disturbed (or "pushed") through a clap, the vibrations of a drumhead, or jet engines ejecting high velocity air. Through our model equations, this initial velocity disturbance perturbs the pressure field and a pressure wave travels with sound speed $c$, until it arrives at our eardrums.  

Although we refer to $p(x,t)$ and $u(x,t)$ as pressure and velocity, these quantities are actually pressure and velocity *perturbations* to a background atmospheric pressure and mean windfield.  With this interpretation, it makes sense to set the initial perturbation $p(x,t)$ to zero.   

For the following questions, assume that we are in an infinite domain $-\infty < x < \infty$, so we do not need to consider even and odd extensions of our initial conditions.

1. Consider an initial pressure field $p(x,0) = 0$, and an initial velocity disturbance given by

\begin{equation*}
u(x,0) = \left\{\begin{array}{rr} 
-1 & -1 < x < 0 \\
1 & 0 < x < 1\\
0 & \mbox{otherwise}
\end{array}
\right.
\end{equation*}

Set $c = \rho = 1$ and sketch (or plot) the solution for $p(x,t)$ and $u(x,t)$ at time $t = 5$.  

2. How does increasing or decreasing the density $\rho$ of the background medium affect the sound that we might hear?  How does increasing the sound speed $c$ in the medium change the sound we hear? 